We can move onto generating the tokenized audio for the GPT2 model. We'll then associate it with the text to feed it in and train it to spit out the same thing.

In [ ]:
from datasets import load_dataset, DatasetDict, Dataset
from mytokenizer import Tokenizer

dataset = load_dataset("blabble-io/libritts_r", "clean", streaming=True)


tokenizer = Tokenizer(checkpoint_dir="checkpoints", sample_rate=24000)


def encode(sample):
    # print(sample)
    text, audio = tokenizer.encode(
        sample["text_normalized"],
        sample["audio"]["array"],
        sample["audio"]["sampling_rate"],
    )
    print(text)

    return {"text": text, "audio": audio}


def remove_too_long(sample):
    return len(sample["text"]) + len(sample["audio"]) <= 200


def pad(sample):
    return {
        "audio": tokenizer.pad(
            sample["audio"], len(sample["text"]) + len(sample["audio"])
        )
    }


dataset = (
    dataset.map(
        encode,
        remove_columns=[
            "text_original",
            "speaker_id",
            "chapter_id",
            "id",
            "path",
            "text_normalized",
        ],
    )
    .filter(remove_too_long)
    .map(pad)
)

samples = []
for i, sample in enumerate(dataset["train.clean.360"]):
    if i >= 10000:
        break
    samples.append(sample)

# Create a Dataset from the collected samples
dataset = Dataset.from_dict(
    {key: [sample[key] for sample in samples] for key in samples[0].keys()}
)

# Convert to DatasetDict
dataset_dict = DatasetDict({"train": dataset})

# Save the dataset to disk
dataset_dict.save_to_disk("preprocessed_dataset")

print("Saved the preprocessed dataset to preprocessed_dataset")


/home/tugdual/miniconda3/envs/xtts/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


✅ Model initialized
Shape of the input : (27, 1, 3000)
Shape of the output of encoder : (27, 64, 375)
Shape of the output of quantizer : (27, 64, 375)
[[[ 0.    0.    0.   ...  0.   -0.25 -0.25]
  [ 0.    0.25  0.25 ...  0.25  0.25  0.25]
  [ 0.    0.    0.   ...  0.25  0.   -0.25]
  ...
  [ 0.5   0.25  0.25 ...  0.25  0.5   0.25]
  [-0.25 -0.25 -0.25 ...  0.   -0.25 -0.25]
  [ 0.    0.    0.   ...  0.25  0.25  0.  ]]

 [[ 0.    0.    0.25 ...  0.   -0.5  -0.25]
  [ 0.25  0.25  0.25 ...  0.5   0.5   0.75]
  [-0.25 -0.25  0.   ...  0.   -0.5  -0.75]
  ...
  [ 0.25  0.25  0.25 ...  0.    0.   -0.75]
  [-0.25 -0.25 -0.25 ...  0.25  0.5   0.75]
  [ 0.    0.    0.25 ...  0.5   0.25  0.5 ]]

 [[ 0.   -0.5   0.5  ... -0.25 -0.5  -0.25]
  [ 0.75  0.5   0.25 ...  0.5   0.75  0.75]
  [-0.5  -0.5   0.   ... -0.25 -0.5  -0.75]
  ...
  [-0.5   0.5   0.5  ... -0.25 -0.25 -0.5 ]
  [ 0.5  -0.25 -0.5  ...  0.5   0.5   0.5 ]
  [ 0.5  -0.5   0.   ...  0.5   0.5   0.25]]

 ...

 [[ 0.    0.25 -0.25 ... -0

TypeError: Cannot concatenate arrays with different numbers of dimensions: got (27, 64, 375), (1,).

: 